In [2]:
import pandas as pd

traindf = pd.read_csv('data/train.csv')
testdf = pd.read_csv('data/test.csv')
traindf['Set'] = 'Train'
testdf['Set'] = 'Test'
togeth = pd.concat([testdf,traindf])
homeplanets = ['Earth', 'Europa', 'Mars']

In [3]:
def splits(dataframe):
    split_df = dataframe['PassengerId'].str.split('_', expand=True)
    dataframe[['Group', 'GroupNumber']] = split_df

    split_df =  dataframe['Cabin'].str.split("/", expand = True)
    dataframe[['Deck', 'Number', 'Side']]= split_df
    
    split_df = dataframe['Name'].str.split(" ",expand = True)
    dataframe[['FristName','LastName']] = split_df


    return dataframe

togeth = splits(togeth)


#### Same name implies home planet

write how many there are beforehand with same last name and therefore home planet 

In [4]:
togeth_cleaned = togeth.dropna(subset=['HomePlanet','LastName'])

# Group by 'LastName' and aggregate HomePlanet into lists
grouped = togeth_cleaned.groupby('LastName')['HomePlanet'].agg(list).reset_index()

# Function to check if all elements in a list are the same
def is_uniform(homeplanets):
    return len(set(homeplanets)) == 1

# Apply the function to identify uniform and mixed groups
grouped['UniformPlanet'] = grouped['HomePlanet'].apply(is_uniform)

# Count how many people are in uniform and mixed planet groups
uniform_groups = grouped[grouped['UniformPlanet'] == True]
mixed_groups = grouped[grouped['UniformPlanet'] == False]

# Sum up the number of people in uniform and mixed groups
people_uniform = uniform_groups['HomePlanet'].apply(len).sum()
people_mixed = mixed_groups['HomePlanet'].apply(len).sum()

print(f"Number of people from groups where all share the same HomePlanet: {people_uniform}")
print(f"Number of people from groups where not all share the same HomePlanet: {people_mixed}")


Number of people from groups where all share the same HomePlanet: 12398
Number of people from groups where not all share the same HomePlanet: 0


#### same group implies homeplanet 

In [5]:
togeth_cleaned = togeth.dropna(subset=['HomePlanet','Group'])


# Group by 'LastName' and aggregate HomePlanet into lists
grouped = togeth_cleaned.groupby('Group')['HomePlanet'].agg(list).reset_index()

# Function to check if all elements in a list are the same
def is_uniform(homeplanets):
    return len(set(homeplanets)) == 1

# Apply the function to identify uniform and mixed groups
grouped['UniformPlanet'] = grouped['HomePlanet'].apply(is_uniform)

# Count how many people are in uniform and mixed planet groups
uniform_groups = grouped[grouped['UniformPlanet'] == True]
mixed_groups = grouped[grouped['UniformPlanet'] == False]

# Sum up the number of people in uniform and mixed groups
people_uniform = uniform_groups['HomePlanet'].apply(len).sum()
people_mixed = mixed_groups['HomePlanet'].apply(len).sum()

print(f"Number of people from groups where all share the same HomePlanet: {people_uniform}")
print(f"Number of people from groups where not all share the same HomePlanet: {people_mixed}")


Number of people from groups where all share the same HomePlanet: 12682
Number of people from groups where not all share the same HomePlanet: 0


## Homeplanet implies deck

In [6]:
def checks(df):
    for home in homeplanets:
        print("\n" + home)
        print(df[df.HomePlanet == home].Deck.value_counts())
    return df

People from Earth are only in decks G, F, E
People from Europa are only in decks B,C,A,D,E,T
people from Mars are only in decks F,E,D

#### Same cabin means same group

In [7]:
togeth_cleaned = togeth.dropna(subset=['Group','Cabin'])


# Cabin by 'LastName' and aggregate Group into lists
Cabined = togeth_cleaned.groupby('Cabin')['Group'].agg(list).reset_index()

# Function to check if all elements in a list are the same
def is_uniform(Groups):
    return len(set(Groups)) == 1

# Apply the function to identify uniform and mixed Cabins
Cabined['UniformPlanet'] = Cabined['Group'].apply(is_uniform)

# Count how many people are in uniform and mixed planet Cabins
uniform_Cabins = Cabined[Cabined['UniformPlanet'] == True]
mixed_Cabins = Cabined[Cabined['UniformPlanet'] == False]

# Sum up the number of people in uniform and mixed Cabins
people_uniform = uniform_Cabins['Group'].apply(len).sum()
people_mixed = mixed_Cabins['Group'].apply(len).sum()

print(f"Number of people from Cabins where all share the same Group: {people_uniform}")
print(f"Number of people from Cabins where not all share the same Group: {people_mixed}")


Number of people from Cabins where all share the same Group: 12671
Number of people from Cabins where not all share the same Group: 0


* persons in the same group all have the same CabinSide, possibly on different CabinDeck
in groups with several CabinDeck, people with no bill are necessarily on CabinDeck:
** Earth on G
** Europa on B
** Mars on E or F
* for Europa groups with several CabinDeck, people with no bill are on deck B

* Children have no bill
* CryoSleep have no bill

In [10]:
togeth_cleaned = togeth.dropna(subset=['Side','Group'])


# Cabin by 'LastName' and aggregate Group into lists
Cabined = togeth_cleaned.groupby('Group')['Side'].agg(list).reset_index()

# Function to check if all elements in a list are the same
def is_uniform(Groups):
    return len(set(Groups)) == 1

# Apply the function to identify uniform and mixed Cabins
Cabined['UniformPlanet'] = Cabined['Side'].apply(is_uniform)

# Count how many people are in uniform and mixed planet Cabins
uniform_Cabins = Cabined[Cabined['UniformPlanet'] == True]
mixed_Cabins = Cabined[Cabined['UniformPlanet'] == False]

# Sum up the number of people in uniform and mixed Cabins
people_uniform = uniform_Cabins['Group'].apply(len).sum()
people_mixed = mixed_Cabins['Group'].apply(len).sum()

print(f"Number of people from Cabins where all share the same Group: {people_uniform}")
print(f"Number of people from Cabins where not all share the same Group: {people_mixed}")


Number of people from Cabins where all share the same Group: 36476
Number of people from Cabins where not all share the same Group: 0


In [20]:
mini = togeth.iloc[:20].copy()

In [25]:
mini[mini.Group == '0013']

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Name,Set,Transported,Group,GroupNumber,Deck,Number,Side,FristName,LastName
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,...,Nelly Carsoning,Test,NaN,0013,01,G,3,S,Nelly,Carsoning


In [34]:
for index,row in togeth.iterrows():
    if pd.isna(row.Side):
        continue
    if len(togeth[togeth.Group == row.Group].dropna(subset = 'Side').Side.unique()) != 1:
        print()
        print(togeth[togeth.Group == row.Group].dropna(subset = 'Side').Side.unique())
        print(row)
        


In [35]:
togeth

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Name,Set,Transported,Group,GroupNumber,Deck,Number,Side,FristName,LastName
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,...,Nelly Carsoning,Test,NaN,0013,01,G,3,S,Nelly,Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,...,Lerome Peckers,Test,NaN,0018,01,F,4,S,Lerome,Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,...,Sabih Unhearfus,Test,NaN,0019,01,C,0,S,Sabih,Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,...,Meratz Caltilter,Test,NaN,0021,01,C,1,S,Meratz,Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,...,Brence Harperez,Test,NaN,0023,01,F,5,S,Brence,Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,...,Gravior Noxnuther,Train,False,9276,01,A,98,P,Gravior,Noxnuther
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,...,Kurta Mondalley,Train,False,9278,01,G,1499,S,Kurta,Mondalley
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,...,Fayey Connon,Train,True,9279,01,G,1500,S,Fayey,Connon
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,...,Celeon Hontichre,Train,False,9280,01,E,608,S,Celeon,Hontichre


In [38]:
togeth['Bills'] = togeth['RoomService'] + togeth['FoodCourt'] + togeth['ShoppingMall'] + togeth['Spa'] + togeth['VRDeck']

In [41]:
togeth[(togeth.Bills == 0) & (togeth.HomePlanet == 'Earth')].Deck

0       G
27      E
29      G
30      G
32      G
       ..
8673    G
8676    G
8680    G
8681    G
8689    G
Name: Deck, Length: 2294, dtype: object

In [37]:
togeth.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Set', 'Transported', 'Group', 'GroupNumber', 'Deck', 'Number',
       'Side', 'FristName', 'LastName', 'Bills'],
      dtype='object')

In [58]:
count = 0

In [59]:
for index,row in togeth.iterrows():
    if len(togeth[togeth.Group == row.Group].dropna(subset = 'Deck').Deck.unique()) != 1:
        if row.Bills == 0 and row.HomePlanet == 'Earth':
            print(index)
            count += 1
            print(row.Deck)
print(count)
      
        


99
nan
147
nan
156
G
157
G
162
G
241
nan
266
G
293
G
300
G
301
G
302
G
304
G
494
G
522
G
523
G
538
G
612
G
677
G
764
G
825
nan
900
G
960
G
992
G
994
G
1014
G
1020
nan
1058
G
1060
G
1076
G
1139
G
1140
G
1154
G
1155
nan
1156
G
1161
G
1226
G
1240
G
1241
G
1312
G
1363
G
1410
G
1411
G
1412
G
1615
G
1659
G
1661
G
1664
G
1667
G
1668
G
1673
G
1674
G
1675
G
1676
G
1678
G
1684
G
1696
G
1711
G
1712
G
1713
G
1716
G
1745
nan
1746
G
1747
G
1749
G
1830
G
1842
G
1853
G
1859
nan
1868
G
1869
G
1904
G
1974
G
1975
G
1990
G
1992
G
2025
G
2027
G
2136
nan
2199
G
2200
G
2221
nan
2301
G
2302
G
2303
G
2305
G
2341
G
2346
nan
2347
G
2402
G
2407
G
2408
G
2410
nan
2431
G
2438
G
2439
G
2440
G
2442
G
2443
G
2483
nan
2505
G
2518
G
2520
G
2521
G
2586
G
2587
G
2588
G
2634
G
2637
G
2670
G
2687
G
2689
G
2717
G
2738
G
2785
G
2820
G
2828
G
2830
G
2874
G
2876
G
2879
G
2880
G
2893
G
2945
G
2951
G
2953
G
2954
G
2956
G
2957
G
2958
G
2972
nan
2987
G
2988
G
2998
G
3090
G
3141
G
3142
G
3160
nan
3180
G
3181
G
3190
G
3192
G
3193
G
3

In [44]:
togeth[togeth.Group == '0158']

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Set,Transported,Group,GroupNumber,Deck,Number,Side,FristName,LastName,Bills
70,0158_01,Earth,False,E/9/S,55 Cancri e,26.0,False,832.0,0.0,0.0,...,Test,NaN,0158,01,E,9,S,Dawne,Jacobbinson,989.0
71,0158_02,Earth,False,F/33/S,TRAPPIST-1e,31.0,False,0.0,0.0,705.0,...,Test,NaN,0158,02,F,33,S,Nelle,Jacobbinson,705.0


In [62]:
togeth[(togeth.Deck == 'E') & (togeth.HomePlanet == 'Earth')]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Set,Transported,Group,GroupNumber,Deck,Number,Side,FristName,LastName,Bills
21,0049_01,Earth,False,E/1/P,TRAPPIST-1e,14.0,False,NaN,0.0,0.0,...,Test,NaN,0049,01,E,1,P,Doria,Flynney,NaN
27,0059_01,Earth,True,E/2/P,TRAPPIST-1e,36.0,False,0.0,0.0,0.0,...,Test,NaN,0059,01,E,2,P,Janne,Carvis,0.0
35,0087_01,Earth,False,E/3/P,TRAPPIST-1e,19.0,False,0.0,20.0,135.0,...,Test,NaN,0087,01,E,3,P,Meremy,Jarvey,787.0
47,0117_01,Earth,False,E/4/P,PSO J318.5-22,24.0,False,0.0,22.0,82.0,...,Test,NaN,0117,01,E,4,P,Megany,Ewiseston,790.0
62,0145_01,Earth,False,E/6/P,TRAPPIST-1e,56.0,False,0.0,44.0,23.0,...,Test,NaN,0145,01,E,6,P,Arlie,Hodes,790.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8538,9118_01,Earth,False,E/585/P,55 Cancri e,13.0,False,0.0,1501.0,23.0,...,Train,True,9118,01,E,585,P,Glenry,Malliamsey,1692.0
8556,9134_01,Earth,False,E/601/S,TRAPPIST-1e,13.0,False,590.0,0.0,32.0,...,Train,False,9134,01,E,601,S,Therly,Leonadorsen,985.0
8591,9170_01,Earth,NaN,E/587/P,TRAPPIST-1e,33.0,False,1060.0,403.0,1.0,...,Train,False,9170,01,E,587,P,Jeandy,Ewins,1609.0
8606,9185_01,Earth,False,E/588/P,TRAPPIST-1e,43.0,False,698.0,94.0,0.0,...,Train,True,9185,01,E,588,P,Elindy,Mckinn,792.0
